**NLP LAB 09**

The best dataset to choose is offensive. Indeed, haste is a dataset about comment that disrespect a social or a group of person, but it can be a point of view, when the offensive dataset is about insult someone.

The best would be taking both dataset.

In [34]:
!pip install datasets
!pip install bertopic
!pip install torch
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [35]:
from datasets import load_dataset
from bertopic import BERTopic
from transformers import AutoTokenizer

In [6]:
dataset = load_dataset("tweet_eval","offensive")
dataset

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #3:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #4:   0%|          | 0/1 [00:00<?, ?obj/s]

...

Generating test split:   0%|          | 0/860 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1324 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /home/victor/.cache/huggingface/datasets/tweet_eval/offensive/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11916
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 860
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1324
    })
})

We see that there is not much value in the test dataset  (About 5.7% of all data) and in the validation dataset (About 8.9% of all data)

In [28]:
dataset["train"]["text"][:5]

['@user Bono... who cares. Soon people will understand that they gain nothing from following a phony celebrity. Become a Leader of your people instead or help and support your fellow countrymen.',
 '@user Eight years the republicans denied obama’s picks. Breitbarters outrage is as phony as their fake president.',
 '@user Get him some line help. He is gonna be just fine. As the game went on you could see him progressing more with his reads. He brought what has been missing. The deep ball presence. Now he just needs a little more time',
 '@user @user She is great. Hi Fiona!',
 "@user She has become a parody unto herself? She has certainly taken some heat for being such an....well idiot. Could be optic too  Who know with Liberals  They're all optics.  No substance"]

In [8]:
dataset["train"][90:95]

{'text': ['@user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user Thank you for adding me Razor!  Patriots- if you aren’t following my Primary account @user - will you take a minute &amp; Follow me? Thanks &amp; MAGA🇺🇸',
  '@user The lies i keep hearing everyday sigh *INHALES* IM TIRED OF YOU PUTTING YOURSELF DOWN AND IF PPL DONT SEE HOW BEAUTIFUL MY BESTFRIEND IS THEN ITS THEIR PROBLEM You are beautiful ive seen iT so its my place to tell you you are beautiful and i accept nothing less. If not more 😤',
  '@user He is very perceptive Sweetie!',
  '@user Dear Paul McFartney-  You had us at Beatles.  You lost us at President Mad Captain.  Burn in hell-  Jon.""',
  '@user you are also the king of taste'],
 'label': [0, 1, 0, 1, 0]}

We notice that all tag is anonymous. They had been replaced by '@user'.

In [30]:
topic_model_train = BERTopic()
topics_train, probs_train = topic_model_train.fit_transform(dataset["train"]["text"])

topic_model_test = BERTopic()
topics_test, probs_test = topic_model_test.fit_transform(dataset["test"]["text"])

topic_model_validation = BERTopic()
topics_validation, probs_validation = topic_model_validation.fit_transform(dataset["validation"]["text"])


In [33]:
from torch import random
random.manual_seed(42)
topic_model_train.get_topic_info()

,Topic,Count,Name
0,-1,4929,-1_is_he_she_to
1,0,1018,0_gun_control_guns_laws
2,1,492,1_maga_qanon_wwg1wga_trump
3,2,420,2_liberals_they_liberal_user
4,3,292,3_antifa_your_user_they
...,...,...,...
121,120,11,120_citizenship_lawyer_country_border
122,121,11,121_bullshit_insanely_chimed_daffy
123,122,10,122_god_army_iook_yahweh
124,123,10,123_china_business_opium_theft


In [31]:
topic_model_test.get_topic_info()

,Topic,Count,Name
0,-1,324,-1_and_the_is_you
1,0,84,0_she_is_the_and
2,1,72,1_user_is_he_she
3,2,67,2_gun_control_user_chicago
4,3,55,3_antifa_the_to_of
5,4,53,4_kavanaugh_liberals_to_the
6,5,51,5_conservatives_liberals_to_are
7,6,32,6_you_are_your_to
8,7,28,7_brexit_eu_the_conservatives
9,8,20,8_you_sales_online_own


In [32]:
topic_model_validation.get_topic_info()

,Topic,Count,Name
0,0,1072,0_user_the_is_to
1,1,129,1_user_antifa_the_to
2,2,123,2_gun_control_user_the


We see there is a difference in topic between the validation, test and train dataset. There will be a risk to train model on some topic, but to test it.

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
